## Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('..../data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов и 21525 строк типы данных (по столбцам): `float64(2)`, `int64(5)`, `object(5)`.

Описание столбцов:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах `days_employed` и `total_income` различается. Значит, в данных есть пропущенные значения.

<div style="border:solid green 1px; padding: 10px">

В каждой строке таблицы — данные о конкретном Клиенте Банка, взявшем кредит. Часть колонок содержит информацию о
самом Клиенте (количество детей, стаж работы, возраст и т.д.). Остальные данные рассказывают о взятом кредите (задолженность
по кредиту, цель получения, ежемесячный доход).
Предварительно можно утверждать, что, данных достаточно для проверки гипотез.

Но встречаются пропуски в данных.
Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Заполнение пропусков

In [3]:
# поиск и подсчет пропусков в столбцах
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
# доля пропущенных значений в каждом из столбцов с пропусками
share_isna_total_income = df['total_income'].isna().sum() / df.shape[0] * 100
share_isna_days_employed = df['days_employed'].isna().sum() / df.shape[0] * 100

print(share_isna_total_income)
share_isna_days_employed

10.099883855981417


10.099883855981417

В двух столбцах есть пропущенные значения.

Пропуски (`NaN`) составляют 10% от общего числа значений, что достаточно много.

Причины возникновения пропусков могут быть технологическими. Такие ошибки возникают при копировании, считывании, смене формата. Сбой может произойти даже во время записи данных.

Заполнить пропуски лучше медианным значением, потому что для анализа количественных переменных важно учитывать, что в данных могут содержаться "выбросы" - сильно заниженные или завышенные, по сравнению с оставльными, данные, а значит среднее будет смещено относительно реального положения вещей. Также не подойдет замена пропусков на ноль или их удаление.


In [5]:
# замена пропусков в 'total_income' на медиану
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(value=total_income_median)

df['total_income'].isna().sum()

0

###  Проверка данных на аномалии и исправления

В столбце `days_employed` есть отрицательное количество дней трудового стажа, что является аномалией (значением, которое не отражает действительность и появилось по какой-то ошибке.), стаж может быть только положительным. 

Возможными причинами появления таких данных могли стать ошибки при расчете формулы для столбца в исходной таблице (к примеру, перепутали местами уменьшаемое и вычитаемое), а также ошибки при ручном копировании данных. 

Обработем аномалии и заполним пропуски в days_employed медианными значениями по этому столбцу.

In [6]:
# приводим стаж к положительному значению
df['days_employed'] = df['days_employed'].abs()
print(df['days_employed'].head())

# замена пропусков в days_employed
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(value=days_employed_median)

#проверка на пропуски
df.isna().sum()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

В столбце `children` также есть отрицательные значения и значение в 20 детей, что, скорее всего является опечаткой, заменим их.

In [7]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [8]:
# приводим значения к положительному
df['children'] = df['children'].abs()

# исправляем опечатку
df.loc[df['children'] == 20, 'children'] = 2

# проверка после исправлений
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Изменение типов данных

Изменим тип данных столбцу `total_income` для дальнейшей работы по категоризации Клиентов в зависимости от дохода.

In [9]:
df['total_income'] = df['total_income'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


###  Удаление дубликатов

Перед поиском дубликатов приведем значения в таблице к одному регистру.

In [10]:
# приведение значений в столбцах к одному регистру
check_list = ['education', 'family_status', 'income_type', 'purpose', 'gender']
for col in df:
    if col in check_list:
        df[col] = df[col].str.lower()

In [11]:
# поиск и подсчет дубликатов
df.duplicated().sum()

71

Методом `.duplicated()` найдены явные дубликаты, данные, значения которых полностью совпадают. Удалим их и проведем проверку, чтобы убедиться, что все отработало верно.

In [12]:
# удаление дубликатов (с удалением с переформированием индексов)
df = df.drop_duplicates().reset_index(drop=True)

# проверка после удаления
df.duplicated().sum()

0

In [13]:
# проверка уникальности значений
df['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

В списке выше есть неявные дубликаты целей, т.е. названия с ошибками или выраженные разными словами.

По факту целей всего четыре:

* *образование*
* *операции с недвижимостью*
* *приобретение автомобиля*
* *проведение мероприятия (свадьба)*

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_purpose()` с двумя параметрами:

`wrong_purpose` — список дубликатов,

`correct_purpose` — строка с правильным значением.

Функция должна исправить колонку `purpose`e в таблице `df`: заменить каждое значение из списка `wrong_purpose` на значение из `correct_purpose`.

In [14]:
# Функция для замены неявных дубликатов
def replace_wrong_purpose(wrong_purpose, correct_purpose):
    for purpose in wrong_purpose:
        df['purpose'] = df['purpose'].replace(wrong_purpose, correct_purpose)
        
# создание переменной верного значения списков значений для замены      
duplicates_auto = ['автомобили', 'автомобиль', 'на покупку автомобиля', 'на покупку подержанного автомобиля',
              'на покупку своего автомобиля', 'свой автомобиль', 'сделка с автомобилем', 'сделка с подержанным автомобилем']
correct_name_auto = 'приобретение автомобиля'
duplicates_edu = ['высшее образование', 'дополнительное образование', 'заняться высшим образованием', 'заняться образованием',
                  'получение высшего образования', 'получение дополнительного образования', 'получение образования', 'профильное образование']
correct_name_edu = 'образование'
duplicates_prop = ['жилье', 'недвижимость', 'операции с жильем', 'операции с коммерческой недвижимостью', 'операции со своей недвижимостью',
                   'покупка жилой недвижимости', 'покупка жилья','покупка жилья для сдачи', 'покупка жилья для семьи',
                   'покупка коммерческой недвижимости', 'покупка недвижимости', 'покупка своего жилья','ремонт жилью',
                  'строительство жилой недвижимости', 'строительство недвижимости', 'строительство собственной недвижимости']
correct_name_prop = 'операции с недвижимостью'
duplicates_wed = ['на проведение свадьбы', 'свадьба', 'сыграть свадьбу']
correct_name_wed = 'проведение мероприятия (свадьба)'

# Вызов функции для устранения неявных дубликатов
replace_wrong_purpose(duplicates_auto, correct_name_auto)
replace_wrong_purpose(duplicates_edu, correct_name_edu)
replace_wrong_purpose(duplicates_prop, correct_name_prop)
replace_wrong_purpose(duplicates_wed, correct_name_wed)

In [15]:
# Проверка на неявные дубликаты
df['purpose'].sort_values().unique()

array(['образование', 'операции с недвижимостью',
       'приобретение автомобиля', 'проведение мероприятия (свадьба)'],
      dtype=object)

In [16]:
# Проверка на явные дубликаты после удаления неявных
df.duplicated().sum()

334

In [17]:
# удаление дубликатов (с удалением с переформированием индексов)
df = df.drop_duplicates().reset_index(drop=True)

# проверка после удаления
df.duplicated().sum()

0

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим две дополнительные таблицы для образования и для семейного положения Клиента.

Новые датафреймы — это "словари", к которым мы сможем обращаться по идентификатору в дальнейшем.

In [18]:
# датафрейм для образования
education_dict = df[['education','education_id']]
education_dict.value_counts()

education            education_id
среднее              1               14876
высшее               0                5212
неоконченное высшее  2                 744
начальное            3                 282
ученая степень       4                   6
dtype: int64

In [19]:
#датафрейм для семецного положения
family_status_dict = df[['family_status','family_status_id']]
family_status_dict.value_counts()

family_status          family_status_id
женат / замужем        0                   12076
гражданский брак       1                    4124
не женат / не замужем  4                    2784
в разводе              3                    1193
вдовец / вдова         2                     943
dtype: int64

Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [20]:
# создание нового датафрейма без двух столбцов
df = df[['children','days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type',
          'debt', 'total_income', 'purpose']]
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,операции с недвижимостью
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,операции с недвижимостью
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,образование
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,проведение мероприятия (свадьба)


###  Категоризация дохода

Для удобства анализа разобъем Клиентов на категории в зависимости от дохода:

* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Для этого напишем фунцию с заданными условиями.

In [21]:
def income__category(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

Протестируем работу функции для каждого правила. Проверим, в какую категорию попадёт Клиент при значениях дохода:

* 25000
* 48000
* 62000
* 204000
* 1100000

In [22]:
print(income__category(25000))
print(income__category(48000))
print(income__category(62000))
print(income__category(204000))
print(income__category(1100000))

E
D
C
B
A


Функция работает корректно. Теперь создадим отдельный столбец с категориями в зависимости от дохода, и в его ячейках запишем значения, возвращаемые функцией.

In [23]:
df['total_income_category'] = df['total_income'].apply(income__category)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,операции с недвижимостью,B
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,операции с недвижимостью,C
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,образование,B
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,проведение мероприятия (свадьба),C
5,0,926.185831,27,0,1,m,компаньон,0,255763,операции с недвижимостью,B
6,0,2879.202052,43,0,0,f,компаньон,0,240525,операции с недвижимостью,B
7,0,152.779569,50,1,0,m,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,f,сотрудник,0,95856,проведение мероприятия (свадьба),C
9,0,2188.756445,41,1,0,m,сотрудник,0,144425,операции с недвижимостью,C


## Проверка гипотез

### Влияние семейного положения Клиента на факт погашения кредита в срок


Соберем данные по возвращаемости кредита в зависимости от семейного положения

In [24]:
# выводим данные по Клиентам, у которых была задолженность по кредиту и группируем по id семейного положения
is_debt = df[df['debt'] == 1]
is_debt.groupby('family_status_id')['debt'].sum()

family_status_id
0    929
1    388
2     63
3     85
4    274
Name: debt, dtype: int64

In [25]:
family_status_dict.value_counts()

family_status          family_status_id
женат / замужем        0                   12076
гражданский брак       1                    4124
не женат / не замужем  4                    2784
в разводе              3                    1193
вдовец / вдова         2                     943
dtype: int64

#### Вывод:

<div style="border:solid green 1px; padding: 10px">

Чаще всего задолженность по погашению кредита встречается в категории `женат / замужем` семейнонго положения (929 случаев) и реже всего в категории `вдовец / вдова` (всего 63 случая), что может говорить о том, что семейное положение влияет на факт возврата кредита в срок.

### Влияние количества детей Клиента на факт погашения кредита в срок

Соберем данные по возвращаемости кредита в зависимости от количества детей

In [26]:
is_debt.pivot_table(index="children", values="debt", aggfunc='sum')

,debt
children,
0,1061
1,445
2,202
3,27
4,4


#### Вывод:

<div style="border:solid green 1px; padding: 10px">

Чем меньше детей у Клиента, тем чаще он не возвращает кредит в срок: при полном отсутствии детей 1061 случай невозврата, а при 5 детях нет ни одного случая образования задолженности.

### Влияние уровня дохода Клиента на факт погашения кредита в срок

Соберем данные по возвращаемости кредита в зависимости от уровня дохода

In [27]:
is_debt.pivot_table(index="total_income_category", values="debt", aggfunc='count')

,debt
total_income_category,
A,2
B,356
C,1358
D,21
E,2


#### Вывод:

<div style="border:solid green 1px; padding: 10px">
Самая невозвратная категория здесь - это люди с доходом от 200 000 до 1 000 000, вторая по невозвратности - это люди, доход которых составляет от 50 000 до 200 000.

### Влияние цели кредита на факт погашения в срок

Соберем данные по возвращаемости кредита в зависимости от его цели

In [28]:
is_debt.pivot_table(index="purpose", values="debt", aggfunc='count')

,debt
purpose,
образование,370
операции с недвижимостью,781
приобретение автомобиля,402
проведение мероприятия (свадьба),186


#### Вывод:

<div style="border:solid green 1px; padding: 10px">

Кредиты под сделки с недвижимостью имеют большее количество просрочек, чем остальные. Реже всего не гасят в срок поженившиеся Клиенты.

## Общий вывод:

<div style="border:solid green 2px; padding: 20px">

В нашем распоряжении были данные Банка о заемщиках. В каждой строке таблицы — данные о конкретном Клиенте Банка, взявшем кредит. Часть колонок содержит информацию о самом Клиенте (количество детей, стаж работы, возраст и т.д.). Остальные данные рассказывают о взятом кредите (задолженность по кредиту, цель получения, ежемесячный доход). 
    
По заказу кредитного отдела Банка мы проверили влияет ли на факт погашения кредита в срок:

1. Семейное положение Клиента:
Чаще всего задолженность по погашению кредита встречается в категории `женат` / `замужем` семейнонго положения (929 случаев) и реже всего в категории `вдовец` / `вдова` (всего 63 случая), что может говорить о том, что семейное положение влияет на факт возврата кредита в срок.
 
2. Количество детей Клиента:
Чем меньше детей у Клиента, тем чаще он не возвращает кредит в срок: при полном отсутствии детей 1061 случай невозврата, а при 5 детях нет ни одного случая образования задолженности.   
    
3. Уpовень дохода Клиента:
Самая невозвратная категория - это люди с доходом от 200 000 до 1 000 000, вторая по невозвратности - это люди, доход которых составляет от 50 000 до 200 000.   

4. Разные цели кредита:
Кредиты под `сделки с недвижимостью` имеют большее количество просрочек, чем остальные. Реже всего не гасят в срок `поженившиеся` Клиенты.    
     
**Таким образом** при формировании модели кредитного скоринга стоит учесть, что:
* состоящим в официальном браке людям тяжелей гасить кредит во время; 
* дети положительно влияют на родителей и те производят выплаты кредитов в положенное время;
* самые "богатые" и самые "бедные" являются и самыми добросовестными кредитоплательщиками;
* от цели кредита зависит его вовзрат в срок, так взятые кредиты на недвижимость вовращаются хуже.    